# Imports

In [2]:
# User-defined functions
# All data preparation functions for GDP data and explanatory variables
from Functions.data_prep import data_GDP, data_explanatory
# Imports for Moran's I analysis and plotting graphs
from Functions.spatial_functions import spatial_weight_matrix, neighborhood_dict_creation, local_moran_density_plot, local_moran_plots_STAC, lisa_cluster_map_STAC, lisa_update_STAC, quadrant_plot_STAC
# Spatiotemporal outlier analysis
from Functions.spatiotemporal_autocorrelation import year_df_creation, even_time_period_GDP, global_STAC_val, local_STAC_per_region, local_STAC, region_df_creation, corr_time_series, deviation_time_series, global_STAC

# All required imports
from esda.moran import Moran_Local
import numpy as np
import pandas as pd
import random
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import time
random.seed(21)

# Data preparation

In [3]:
# Data preparation and cleaning for GDP data
# Setting time_period option to all years since this is spatiotemporal analysis and all time periods needed
time_period_option = 'all_years'
NUTS_level = 2
gdf_lvl = data_GDP(NUTS_level, time_period_option)

In [4]:
# Making sure that each region has the same number of time periods which is required for STAC Analysis!
gdf_lvl = even_time_period_GDP(gdf_lvl)

# Local STAC calculation

In [5]:
## Extract only for one year to calculate spatial weight (preventing repeated regions)!!
region_df = year_df_creation(gdf_lvl, 2004)
region_df = region_df.drop(columns=['TIME_PERIOD'])
## Making sure that the index of region is NUTS_ID
region_df.set_index('NUTS_ID', inplace=True)

# Spatial Weights Creation
w_adaptive = spatial_weight_matrix(region_df, 15)
neighbor_weights_dict = neighborhood_dict_creation(w_adaptive)

In [6]:
local_STAC_dict = local_STAC(neighbor_weights_dict, gdf_lvl, region_df, 'GDP_VALUE')

In [7]:
# Remember to reset index before updating it with lisa values and corresponding significance and quadrant values
region_df.reset_index(inplace=True)

In [8]:
# Updating the dataframe to include local LISA information
variable = "GDP_VALUE"
lisa_results = lisa_update_STAC(region_df, gdf_lvl, local_STAC_dict, w_adaptive, variable)
# Extracting the results
region_df = lisa_results[0]
quadrant_df = lisa_results[1]

In [9]:
final_df = gdf_lvl.sort_values(by=['NUTS_ID', 'TIME_PERIOD'])

# Statistical Significance Testing

### Global STAC Permutations

In [10]:
variable = 'GDP_VALUE'
denominator = 0.0
# Group by TIME_PERIOD and calculate the mean GDP value summed over all regions for each year
mean_gdp_sum_per_year = final_df.groupby('TIME_PERIOD')[variable].mean()
# Convert the resulting series to a DataFrame
mean_gdp_df = mean_gdp_sum_per_year.to_frame(name='Mean_variable')
mean_gdp_array = mean_gdp_df['Mean_variable']
# Calculate denominator (same for each region)
for region, neighbor_weights in neighbor_weights_dict.items():
    time_series_i = region_df_creation(final_df, region)
    corr_i = corr_time_series(time_series_i[variable], mean_gdp_array)
    dev_i = deviation_time_series(corr_i, time_series_i[variable], mean_gdp_array)
    dev_i_squared = dev_i ** 2
    denominator += dev_i_squared
denominator

835152411781.0594

In [11]:
# Initializing variables
spatial_w_sum = 0.0
# Calculate sum of all spatial weights
# Iterate over all regions
for region, neighbor_weights in neighbor_weights_dict.items():
    # Iterate over all neighbor weights for the current region
    for weight in neighbor_weights.values():
        # Add the weight to the total sum
        spatial_w_sum += weight
# Calculate total number of regions
N = len(region_df)
first_term = N / spatial_w_sum

In [12]:
def global_randomization(df, num_simulations=999, num_jobs=-1):
    regions = df['NUTS_ID'].unique()

    def run_simulation(i):
        # Create a randomized dataset by shuffling the entire time series across regions
        shuffled_df = df.copy()
        shuffled_regions = np.random.permutation(regions)
        print(shuffled_regions)
        # Mapping of original regions to shuffled regions
        region_mapping = {original: shuffled for original, shuffled in zip(regions, shuffled_regions)}
        # Apply the new mapping to the dataframe
        shuffled_df['NUTS_ID'] = shuffled_df['NUTS_ID'].map(region_mapping)
        shuffled_df = shuffled_df.sort_values(by=['NUTS_ID', 'TIME_PERIOD']).reset_index(drop=True)
        # Calculate global LISA values for the shuffled dataset
        global_STAC = global_STAC_val(neighbor_weights_dict, shuffled_df, variable, mean_gdp_array, first_term)

        return global_STAC

    print(f"Starting {num_simulations} simulations...")
    start_time = time.time()
    results = Parallel(n_jobs=num_jobs)(delayed(run_simulation)(i) for i in range(num_simulations))
    end_time = time.time()
    print(f"All simulations completed in {end_time - start_time:.2f} seconds")
    return np.array(results)

# Running with print statements to monitor progress
simulated_global_STACs = global_randomization(gdf_lvl, num_simulations=999, num_jobs=12)

Starting 999 simulations...
All simulations completed in 1530.54 seconds


#### Global STAC p-value

In [14]:
# Calculate global STAC value
original_global_STAC = global_STAC(neighbor_weights_dict, gdf_lvl, region_df, 'GDP_VALUE')
original_global_STAC

0.6207243325948555

In [15]:
# Calculate the p-value
p_value = np.mean(simulated_global_STACs >= original_global_STAC)

print(f"Original Global LISA: {original_global_STAC}")
print(f"P-value: {p_value}")

Original Global LISA: 0.6207243325948555
P-value: 0.0


### LISA Permutations

WARNING: The code below takes 12 hours to run due to the high number of permutations

In [ ]:
def conditional_randomization(df, num_simulations=999, num_jobs=-1):
    regions = df['NUTS_ID'].unique()
    n = len(regions)
    region_index_map = {region: idx for idx, region in enumerate(regions)}
    # Precompute fixed series for each region to avoid repeated filtering
    fixed_series_dict = {region: df[df['NUTS_ID'] == region] for region in regions}
    # Precompute remaining series for each region
    remaining_series_dict = {region: df[df['NUTS_ID'] != region] for region in regions}
    def run_simulation(i):
        simulated_I = np.zeros(n)
        for region in regions:
            fixed_series = fixed_series_dict[region]
            remaining_regions = remaining_series_dict[region]
            shuffled_regions = np.random.permutation(remaining_regions['NUTS_ID'].unique())
            # List to collect DataFrames and concatenate once
            df_list = [fixed_series.copy()]
            for original_region, shuffled_region in zip(regions[regions != region], shuffled_regions):
                shuffled_series = remaining_regions[remaining_regions['NUTS_ID'] == shuffled_region].copy()
                shuffled_series['NUTS_ID'] = original_region
                df_list.append(shuffled_series)
            randomized_df = pd.concat(df_list, ignore_index=True)
            randomized_df = randomized_df.sort_values(by=['NUTS_ID', 'TIME_PERIOD']).reset_index(drop=True)
            simulated_I[region_index_map[region]] = local_STAC_per_region(neighbor_weights_dict, randomized_df, region_df, variable, region, denominator, mean_gdp_array)
        return simulated_I

    print(f"Starting {num_simulations} simulations...")
    start_time = time.time()
    results = Parallel(n_jobs=num_jobs)(delayed(run_simulation)(i) for i in range(num_simulations))
    end_time = time.time()
    print(f"All simulations completed in {end_time - start_time:.2f} seconds")
    return np.array(results)

# Assuming final_df, neighbor_weights_dict, and region_df are defined
# Running with print statements to monitor progress
simulated_LISAs = conditional_randomization(final_df, num_simulations=999, num_jobs=12)

Starting 999 simulations...


C:\Users\20211147\PycharmProjects\DataMiningCodes\venv\lib\site-packages\joblib\externals\loky\process_executor.py:700: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


#### Saving LISA Permutations

In [22]:
# You can optionally provide column names, here we'll use generic names
column_names = [f'Column{i+1}' for i in range(simulated_LISAs.shape[1])]
df = pd.DataFrame(simulated_LISAs, columns=column_names)

# Step 4: Save the DataFrame to a CSV file
df.to_csv('results/simulated_LISAs_STAC_final.csv', index=False)

#### LISA P-value calculation

In [19]:
# Function to calculate p-values, mean, and variance
def calculate_statistics(observed_Is, simulated_Is):
    num_simulations = simulated_Is.shape[0]
    # Calculate p-values for positive and negative spatial autocorrelation
    p_values_positive = np.sum(simulated_Is >= observed_Is, axis=0) / num_simulations
    p_values_negative = np.sum(simulated_Is <= observed_Is, axis=0) / num_simulations
    # Combine p-values for two-sided test
    p_values = np.minimum(p_values_positive, p_values_negative) * 2
    empirical_means = np.mean(simulated_Is, axis=0)
    empirical_vars = np.var(simulated_Is, axis=0)
    z_scores = (observed_Is - empirical_means) / np.sqrt(empirical_vars)

    return p_values, empirical_means, empirical_vars, z_scores

In [27]:
# Extracting saved simulated LISAs
simulated_LISA_df = pd.read_csv('results/simulated_LISAs_STAC.csv')
# Changing column names to resemble LISA values
regions = list(final_df['NUTS_ID'].unique())
simulated_LISA_df.columns = regions
# Converting data types of input variables for statistics calculation function
# Convert dictionary values to a numpy array
observed_Is = np.array(list(local_STAC_dict.values()))
simulated_Is = simulated_LISA_df.values
# Calling the statistic function
# Example usage of calculate_statistics
p_values, empirical_means, empirical_vars, z_scores = calculate_statistics(observed_Is, simulated_Is)

In [42]:
# Adding p-values to final LISA_df
# Creating a copy to prevent confusion when slicing
region_df_copy = region_df.copy()
# Create a DataFrame with the extracted p-values
p_values_df = pd.DataFrame(p_values, columns=['LISA_p_values'])
region_df_copy['LISA_p_values'] = p_values_df['LISA_p_values']
# Adding a significance column to indicate LISAs below 0.05
region_df_copy['LISA_significant'] = region_df_copy['LISA_p_values'].apply(lambda x: 'Significant' if x < 0.05 else 'Non-Significant')

In [44]:
# Saving the new updated LISA_df since it now has significant LISA regions
region_df_copy.to_csv("results/STAC_LISA_df.csv")